In [21]:
import pandas as pd

In [22]:
# Load the CSV file into a DataFrame bolig_laeger_st_DST
csv_file_path = 'outputdata/NW_Merged.csv'   # Ret sti
data = pd.read_csv(csv_file_path , index_col=1)

data_imputed = data[[
    'Rooms_imputed',
    'Toilets_imputed',
    'Karakter_imputed',
    'indikator_imputed',
    'Socioeconomic_Ref_Imputed',
    'sum_imputed'
]]

data_for_model=data.drop(['Schoolname', 
                        'Average_Grade',
                        'Indikatorsvar',
                        'Matched_Kommune',
                        'link', 
                        'Unnamed: 0.1',
                        'Rooms_imputed',
                        'Toilets_imputed',
                        'Karakter_imputed',
                        'indikator_imputed',
                        'Socioeconomic_Ref_Imputed',
                        'sum_imputed', 
                        'Postal code'
                                        
                                          ], axis=1)


In [23]:
#Data for landkort

data_nodup = data[~data.index.duplicated(keep='first')]
house_count_df = data_nodup.groupby('Postal code').size().reset_index(name='NumberOfHouses')
house_count_df.to_csv('outputdata/Post_count.csv', index=False)

In [24]:
pd.set_option('display.max_column', None)
data_uden_missing = data_for_model.dropna()
data_uden_missing = data_uden_missing[data_uden_missing['Boligtype'].str.strip() != 'Fritidsbolig']
data_uden_missing.to_csv('outputdata/data_uden_missing.csv', index=False)

In [25]:
import pandas as pd

def create_dummies_with_reference(df, columns_with_references):
    """
    Creates dummy variables with a specified reference category.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    columns_with_references (dict): A dictionary where keys are column names and values are the reference categories.

    Returns:
    pd.DataFrame: DataFrame with dummy variables created, with the specified reference categories dropped.
    """
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original

    for column, reference in columns_with_references.items():
        # Ensure the column is categorical
        df[column] = df[column].astype('category')
        
        # Check if the reference exists in the column
        if reference not in df[column].cat.categories:
            raise ValueError(f"The reference category '{reference}' does not exist in the column '{column}'")
        
        # Reorder categories to place the reference first
        df[column] = df[column].cat.reorder_categories(
            [reference] + [cat for cat in df[column].cat.categories if cat != reference], 
            ordered=True
        )
    
    # Generate dummies with the first category (the reference) dropped
    df = pd.get_dummies(df, columns=columns_with_references.keys(), drop_first=True)
    
    return df

# Example usage:
columns_with_references = {
    'Boligtype': 'Villa',
    'Energy-classification':' D',
    'Radon': 'Mellem',
    'Kommune': 'herning kommune',
    '1 årig - Socioøkonomisk Reference Forskel': 'På Niveau'
}

# Apply the function
data_uden_missing = create_dummies_with_reference(data_uden_missing, columns_with_references)

In [26]:
"""
data_uden_missing = pd.get_dummies(data_uden_missing, columns=['Boligtype', 'Energy-classification', 'Radon', 
                                                  'Kommune', '1 årig - Socioøkonomisk Reference Forskel'], drop_first=True)
"""
data_uden_missing

,Price,Distance to school,Trivsel,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Elevator,Altan,Terasse,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand,Antal_laeger,Stationer,Karakter,Boligtype_Ejerlejlighed,Boligtype_Landejendom,Boligtype_Rækkehus,Boligtype_Villalejlighed,Energy-classification_ A,Energy-classification_ A2,Energy-classification_ A2010,Energy-classification_ A2015,Energy-classification_ A2020,Energy-classification_ B,Energy-classification_ C,Energy-classification_ E,Energy-classification_ F,Energy-classification_ G,Energy-classification_fejl,Radon_Høj,Radon_Lav,Radon_Meget høj,Radon_Meget lav,Radon_Ukendt,Kommune_aabenraa kommune,Kommune_aalborg kommune,Kommune_aarhus kommune,Kommune_albertslund kommune,Kommune_allerød kommune,Kommune_assens kommune,Kommune_ballerup kommune,Kommune_billund kommune,Kommune_bornholm kommune,Kommune_brøndby kommune,Kommune_brønderslev kommune,Kommune_dragør kommune,Kommune_egedal kommune,Kommune_esbjerg kommune,Kommune_faaborg-midtfyn kommune,Kommune_fanø kommune,Kommune_favrskov kommune,Kommune_faxe kommune,Kommune_fredensborg kommune,Kommune_fredericia kommune,Kommune_frederiksberg kommune,Kommune_frederikshavn kommune,Kommune_frederikssund kommune,Kommune_furesø kommune,Kommune_gentofte kommune,Kommune_gladsaxe kommune,Kommune_glostrup kommune,Kommune_greve kommune,Kommune_gribskov kommune,Kommune_guldborgsund kommune,Kommune_haderslev kommune,Kommune_halsnæs kommune,Kommune_hedensted kommune,Kommune_helsingør kommune,Kommune_herlev kommune,Kommune_hillerød kommune,Kommune_hjørring kommune,Kommune_holbæk kommune,Kommune_holstebro kommune,Kommune_horsens kommune,Kommune_hvidovre kommune,Kommune_høje-taastrup kommune,Kommune_hørsholm kommune,Kommune_ikast-brande kommune,Kommune_ishøj kommune,Kommune_jammerbugt kommune,Kommune_kalundborg kommune,Kommune_kerteminde kommune,Kommune_kolding kommune,Kommune_københavns kommune,Kommune_køge kommune,Kommune_langeland kommune,Kommune_lejre kommune,Kommune_lemvig kommune,Kommune_lolland kommune,Kommune_lyngby-taarbæk kommune,Kommune_læsø kommune,Kommune_mariagerfjord kommune,Kommune_middelfart kommune,Kommune_morsø kommune,Kommune_norddjurs kommune,Kommune_nordfyns kommune,Kommune_nyborg kommune,Kommune_næstved kommune,Kommune_odder kommune,Kommune_odense kommune,Kommune_odsherred kommune,Kommune_randers kommune,Kommune_rebild kommune,Kommune_ringkøbing-skjern kommune,Kommune_ringsted kommune,Kommune_roskilde kommune,Kommune_rudersdal kommune,Kommune_rødovre kommune,Kommune_samsø kommune,Kommune_silkeborg kommune,Kommune_skanderborg kommune,Kommune_skive kommune,Kommune_slagelse kommune,Kommune_solrød kommune,Kommune_sorø kommune,Kommune_stevns kommune,Kommune_struer kommune,Kommune_svendborg kommune,Kommune_syddjurs kommune,Kommune_sønderborg kommune,Kommune_thisted kommune,Kommune_tårnby kommune,Kommune_tønder kommune,Kommune_vallensbæk kommune,Kommune_varde kommune,Kommune_vejen kommune,Kommune_vejle kommune,Kommune_vesthimmerlands kommune,Kommune_viborg kommune,Kommune_vordingborg kommune,Kommune_ærø kommune,1 årig - Socioøkonomisk Reference Forskel_Bedre end forventet,1 årig - Socioøkonomisk Reference Forskel_Dårligere end forventet
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Mellemtoftevej 36,2600",3995000.0,0.391,3.502026,24.60,25.00,1.0,128.0,388.0,2.889,5.0,1947,1.0,0,0,0,2660,1096,407100.0,47.4,23899.0,0.738859,0.055232,0.205908,13.3,6.0,0.0,6.896407,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [27]:
non_numeric_columns = data_uden_missing.select_dtypes(include=['object']).columns
non_numeric_columns

# Force all columns to numeric, coercing errors to NaN
data_numeric = data_uden_missing.apply(pd.to_numeric, errors='coerce')

# Identify columns that could not be converted to numeric and remain non-numeric
non_numeric_cols = data_numeric.columns[data_numeric.dtypes == 'object']

# Display any non-numeric columns
non_numeric_cols

data_numeric = data_numeric.astype('float32')


In [28]:
data_numeric.to_csv('outputdata/data_ready.csv')
data_numeric

,Price,Distance to school,Trivsel,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Elevator,Altan,Terasse,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand,Antal_laeger,Stationer,Karakter,Boligtype_Ejerlejlighed,Boligtype_Landejendom,Boligtype_Rækkehus,Boligtype_Villalejlighed,Energy-classification_ A,Energy-classification_ A2,Energy-classification_ A2010,Energy-classification_ A2015,Energy-classification_ A2020,Energy-classification_ B,Energy-classification_ C,Energy-classification_ E,Energy-classification_ F,Energy-classification_ G,Energy-classification_fejl,Radon_Høj,Radon_Lav,Radon_Meget høj,Radon_Meget lav,Radon_Ukendt,Kommune_aabenraa kommune,Kommune_aalborg kommune,Kommune_aarhus kommune,Kommune_albertslund kommune,Kommune_allerød kommune,Kommune_assens kommune,Kommune_ballerup kommune,Kommune_billund kommune,Kommune_bornholm kommune,Kommune_brøndby kommune,Kommune_brønderslev kommune,Kommune_dragør kommune,Kommune_egedal kommune,Kommune_esbjerg kommune,Kommune_faaborg-midtfyn kommune,Kommune_fanø kommune,Kommune_favrskov kommune,Kommune_faxe kommune,Kommune_fredensborg kommune,Kommune_fredericia kommune,Kommune_frederiksberg kommune,Kommune_frederikshavn kommune,Kommune_frederikssund kommune,Kommune_furesø kommune,Kommune_gentofte kommune,Kommune_gladsaxe kommune,Kommune_glostrup kommune,Kommune_greve kommune,Kommune_gribskov kommune,Kommune_guldborgsund kommune,Kommune_haderslev kommune,Kommune_halsnæs kommune,Kommune_hedensted kommune,Kommune_helsingør kommune,Kommune_herlev kommune,Kommune_hillerød kommune,Kommune_hjørring kommune,Kommune_holbæk kommune,Kommune_holstebro kommune,Kommune_horsens kommune,Kommune_hvidovre kommune,Kommune_høje-taastrup kommune,Kommune_hørsholm kommune,Kommune_ikast-brande kommune,Kommune_ishøj kommune,Kommune_jammerbugt kommune,Kommune_kalundborg kommune,Kommune_kerteminde kommune,Kommune_kolding kommune,Kommune_københavns kommune,Kommune_køge kommune,Kommune_langeland kommune,Kommune_lejre kommune,Kommune_lemvig kommune,Kommune_lolland kommune,Kommune_lyngby-taarbæk kommune,Kommune_læsø kommune,Kommune_mariagerfjord kommune,Kommune_middelfart kommune,Kommune_morsø kommune,Kommune_norddjurs kommune,Kommune_nordfyns kommune,Kommune_nyborg kommune,Kommune_næstved kommune,Kommune_odder kommune,Kommune_odense kommune,Kommune_odsherred kommune,Kommune_randers kommune,Kommune_rebild kommune,Kommune_ringkøbing-skjern kommune,Kommune_ringsted kommune,Kommune_roskilde kommune,Kommune_rudersdal kommune,Kommune_rødovre kommune,Kommune_samsø kommune,Kommune_silkeborg kommune,Kommune_skanderborg kommune,Kommune_skive kommune,Kommune_slagelse kommune,Kommune_solrød kommune,Kommune_sorø kommune,Kommune_stevns kommune,Kommune_struer kommune,Kommune_svendborg kommune,Kommune_syddjurs kommune,Kommune_sønderborg kommune,Kommune_thisted kommune,Kommune_tårnby kommune,Kommune_tønder kommune,Kommune_vallensbæk kommune,Kommune_varde kommune,Kommune_vejen kommune,Kommune_vejle kommune,Kommune_vesthimmerlands kommune,Kommune_viborg kommune,Kommune_vordingborg kommune,Kommune_ærø kommune,1 årig - Socioøkonomisk Reference Forskel_Bedre end forventet,1 årig - Socioøkonomisk Reference Forskel_Dårligere end forventet
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Mellemtoftevej 36,2600",3995000.0,0.391,3.502026,24.600000,25.000000,1.0,128.0,388.0,2.889,5.0,1947.0,1.0,0.0,0.0,0.0,2660.0,1096.0,407100.0,47.400002,23899.0,0.738859,0.055232,0.205908,13.300000,6.0,0.0,6.896407,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [29]:
#OLS
import statsmodels.api as sm

# Redefine the dependent and independent variables
Y = data_numeric['Price']
X = data_numeric.drop(columns=['Price'])

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Perform OLS regression with robust standard errors
model = sm.OLS(Y, X)
results = model.fit(cov_type='HC3')  # HC3 provides robust standard errors

# Display the regression summary
ols_summary_clean = results.summary()
ols_summary_clean


/Applications/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 145, but rank is 132
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     87.71
Date:                Thu, 22 Aug 2024   Prob (F-statistic):               0.00
Time:                        11:38:28   Log-Likelihood:            -3.1867e+05
No. Observations:               19848   AIC:                         6.376e+05
Df Residuals:                   19713   BIC:                         6.387e+05
Df Model:                         134                                         
Covariance Type:                  HC3                                         
=====================================================================================================================================
                                                                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------
const                                                             -4.682e+05   7.16e+04     -6.538      0.000   -6.09e+05   -3.28e+05
Distance to school                                                -8.166e+04   7310.186    -11.171      0.000    -9.6e+04   -6.73e+04
Trivsel                                                            3.288e+06   2.22e+05     14.791      0.000    2.85e+06    3.72e+06
Kommuneskat                                                        -5.71e+05    4.6e+04    -12.416      0.000   -6.61e+05   -4.81e+05
Grundskyldspromillen                                               6.305e+04   5212.585     12.095      0.000    5.28e+04    7.33e+04
Liggetid                                                           -386.4616    108.019     -3.578      0.000    -598.176    -174.748
m2                                                                 2.236e+04   1384.949     16.146      0.000    1.96e+04    2.51e+04
Grund                                                                21.1251      2.264      9.330      0.000      16.687      25.563
Ejerudgift                                                          817.9792     90.144      9.074      0.000     641.299     994.659
Rooms                                                             -1.274e+04   3.21e+04     -0.397      0.691   -7.56e+04    5.01e+04
Build                                                              -919.4988    482.286     -1.907      0.057   -1864.762      25.764
Toilets                                                             6.27e+05   7.24e+04      8.655      0.000    4.85e+05    7.69e+05
Elevator                                                           1.459e+05   1.32e+05      1.104      0.269   -1.13e+05    4.05e+05
Altan                                                              -1.95e+05   9.01e+04     -2.165      0.030   -3.71e+05   -1.85e+04
Terasse                                                            8.242e+05    2.8e+05      2.940      0.003    2.75e+05    1.37e+06
antal anmeldelser                                                   682.6332     36.099     18.910      0.000     611.881     753.385
antal sigtelser                                                   -1888.0299    105.907    -17.827      0.000   -2095.604   -1680.456
Familiernes gennemsnitlige indkomst indkomst                          7.5777      0.530     14.293      0.000       6.539       8.617
m2 parker, sportsanlæg og rekreative omr pr indbygger             -1.096e+04    939.694    -11.665      0.000   -1.28e+04   -9119.904
indbyggertal                                                         -4.1347      0.899     -4.598      0.000      -5.897      -2.372
a